In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
dogecoin = pd.read_csv("../data/doge_daily.tsv", sep='\t')

In [3]:
dogecoin.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Apr 15, 2021",0.1212,0.1873,0.1207,0.1822,0.1822,"17,916,620,386"
1,"Apr 14, 2021",0.0943,0.1425,0.0925,0.1215,0.1215,"20,977,679,327"
2,"Apr 13, 2021",0.0707,0.0952,0.0707,0.0934,0.0934,"7,176,815,089"
3,"Apr 12, 2021",0.0745,0.0750,0.0689,0.0708,0.0708,"2,687,720,380"
4,"Apr 11, 2021",0.0637,0.0792,0.0632,0.0746,0.0746,"6,646,627,317"


In [4]:
dogecoin['Date'] = pd.to_datetime(dogecoin['Date'])

In [5]:
dogecoin = dogecoin.set_index('Date').sort_index().reset_index()

In [6]:
dogecoin.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,2014-09-17,0.0003,0.0003,0.0003,0.0003,0.0003,"1,463,600"
1,2014-09-18,0.0003,0.0003,0.0003,0.0003,0.0003,"2,215,910"
2,2014-09-19,0.0003,0.0003,0.0003,0.0003,0.0003,"883,563"
3,2014-09-20,0.0003,0.0003,0.0003,0.0003,0.0003,"993,004"
4,2014-09-21,0.0003,0.0003,0.0003,0.0003,0.0003,"539,140"


In [7]:
dogecoin = dogecoin.rename(columns={'Close*' : 'Close'})

In [8]:
dogecoin['Daily_Change'] = dogecoin['Close'] - dogecoin['Open']
dogecoin['Daily_Change_Ind'] = np.where(dogecoin['Daily_Change'] > 0, 1, 0)

In [9]:
dogecoin.head()

,Date,Open,High,Low,Close,Adj Close**,Volume,Daily_Change,Daily_Change_Ind
0,2014-09-17,0.0003,0.0003,0.0003,0.0003,0.0003,"1,463,600",0.0,0
1,2014-09-18,0.0003,0.0003,0.0003,0.0003,0.0003,"2,215,910",0.0,0
2,2014-09-19,0.0003,0.0003,0.0003,0.0003,0.0003,"883,563",0.0,0
3,2014-09-20,0.0003,0.0003,0.0003,0.0003,0.0003,"993,004",0.0,0
4,2014-09-21,0.0003,0.0003,0.0003,0.0003,0.0003,"539,140",0.0,0


In [10]:
def expo_moving_average(df, period):
    '''
    Calculate the exponential moving average.
    
    Input:
        df: a pandas dataframe
        period (int): time period
    Returns:
        rv: a pd series of exponential moving averages
    '''
    col_name = f'EMA_{period}' 
    rv = pd.Series([None] * df.shape[0])
    weight_0 = 2 / (period + 1) 
    weight_1 = 1 - weight_0
    for row in df.itertuples():
        if row.Index < period - 1:
            continue
        elif row.Index == period - 1:
            simple_average = df.loc[:period - 1, 'Close'].sum() / period
            #df.loc[row.Index, col_name] = simple_average
            rv[row.Index] = simple_average
        else:
            #ema = row.Close * weight_0 + df.loc[row.Index - 1, col_name] * weight_1
            ema = row.Close * weight_0 + rv[row.Index - 1] * weight_1
            #df.loc[row.Index, col_name] = ema
            rv[row.Index] = ema
    return rv

In [11]:
def moving_average_convergence_divergence(df):
    '''
    Find the moving average convergence divergence, which is equal to the
    difference between the 12-period exponential moving average and
    26-period exponential moving average.
    
    Modifies the dataframe in place.
    
    Input:
        df: a pd dataframe
    '''
    col_name = 'MACD'
    ema_12 = expo_moving_average(dogecoin, 12)
    ema_26 = expo_moving_average(dogecoin, 26)
    df['MACD'] = ema_12 - ema_26

In [12]:
moving_average_convergence_divergence(dogecoin)

In [13]:
def proc(df, period):
    '''
    Calculate the price rate of change.
    
    Input:
        df: a pandas dataframe
        period (int): time period
    '''
    
    col_name = f'PROC_{period}'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < period:
            continue
        prev_idx = row.Index - period
        df.loc[row.Index, col_name] = (df.loc[row.Index, 'Close'] - df.loc[prev_idx, 'Close']) / df.loc[prev_idx, 'Close']

In [14]:
proc(dogecoin, 3)
proc(dogecoin, 5)
proc(dogecoin, 10)

In [15]:
def williams_percentage_range(df):
    '''
    Find the Williams Percentage Range. Modifies the df in place. 
    
    Input:
        df: a pd dataframe
    '''
    col_name = 'wpr'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < 14:
            continue
        max_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].max()
        min_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].min()
        try:
            wpr = (max_14 - row.Close) / (max_14 - min_14) * -100
        except:
            wpr = 0
        df.loc[row.Index, col_name] = wpr

In [16]:
williams_percentage_range(dogecoin)

In [17]:
def stochastic_oscillator(df):
    '''
    Find the stochastic oscillator. Modifies the df in place.
    '''
    col_name = 'sto_os'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < 14:
            continue
        max_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].max()
        min_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].min()
        try:
            so = (row.Close - min_14) / (max_14 - min_14) * 100
        except:
            so = 0
        df.loc[row.Index, col_name] = so

In [18]:
stochastic_oscillator(dogecoin)

In [19]:
dogecoin.head(30)

,Date,Open,High,Low,Close,Adj Close**,Volume,Daily_Change,Daily_Change_Ind,MACD,PROC_3,PROC_5,PROC_10,wpr,sto_os
0,2014-09-17,0.0003,0.0003,0.0003,0.0003,0.0003,"1,463,600",0.0000,0,NaN,None,None,None,None,None
1,2014-09-18,0.0003,0.0003,0.0003,0.0003,0.0003,"2,215,910",0.0000,0,NaN,None,None,None,None,None
2,2014-09-19,0.0003,0.0003,0.0003,0.0003,0.0003,"883,563",0.0000,0,NaN,None,None,None,None,None
3,2014-09-20,0.0003,0.0003,0.0003,0.0003,0.0003,"993,004",0.0000,0,NaN,0.0,None,None,None,None
4,2014-09-21,0.0003,0.0003,0.0003,0.0003,0.0003,"539,140",0.0000,0,NaN,0.0,None,None,None,None
5,2014-09-22,0.0003,0.0003,0.0003,0.0003,0.0003,"620,222",0.0000,0,NaN,0.0,0.0,None,None,None
6,2014-09-23,0.0003,0.0003,0.0003,0.0003,0.0003,"739,197",0.0000,0,NaN,0.0,0.0,None,None,None
7,2014-09-24,0.0003,0.0004,0.0003,0.0003,0.0003,"1,277,840",0.0000,0,NaN,0.0,0.0,None,None,None
8,2014-09-25,0.0003,0.0004,0.0003,0.0004,0.0004,"2,393,610",0.0001,1,NaN,0.333333,0.333333,None,None,None
9,2014-09-26,0.0004,0.0005,0.0004,0.0005,0.0005,"4,722,610",0.0001,1,NaN,0.666667,0.666667,None,None,None


In [20]:
dogecoin.to_csv('../data/doge_daily_modified.csv')